### LCEL Deepdive

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

In [ ]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

In [ ]:
prompt.invoke({"topic": "ice cream"})

In [ ]:
from langchain_core.messages.human import HumanMessage

messages = [HumanMessage(content='tell me a short joke about ice cream')]
model.invoke(messages)

In [ ]:
from langchain_core.messages import AIMessage

ai_msg = AIMessage(content='Why did the ice cream truck break down? It had too many "scoops"!')

output_parser.invoke(ai_msg)

### Operator Overloading

In [ ]:
print(2+4)

In [ ]:
result = (2).__add__(4)

print(result)

In [17]:
class StupidAdder:
    def __init__(self, number):
        self.number = number

    def __add__(self, other):
        return StupidAdder(self.number + other.number + 42)

    def __str__(self):
        return str(self.number)

In [ ]:
first = StupidAdder(5)
second = StupidAdder(10)

print(first + second)

### What is this "|" in Python?

In [20]:
from abc import ABC, abstractmethod

class CRunnable(ABC):
    def __init__(self):
        self.next = None

    @abstractmethod
    def process(self, data):
        """
        This method must be implemented by subclasses to define
        data processing behavior.
        """
        pass

    def invoke(self, data):
        processed_data = self.process(data)
        if self.next is not None:
            return self.next.invoke(processed_data)
        return processed_data

    def __or__(self, other):
        return CRunnableSequence(self, other)

class CRunnableSequence(CRunnable):
    def __init__(self, first, second):
        super().__init__()
        self.first = first
        self.second = second

    def process(self, data):
        pass

    def invoke(self, data):
        first_result = self.first.invoke(data)
        return self.second.invoke(first_result)



In [21]:
class AddTen(CRunnable):
    def process(self, data):
        print("AddTen: ", data)
        return data + 10

class MultiplyByTwo(CRunnable):
    def process(self, data):
        print("Multiply by 2: ", data)
        return data * 2

class ConvertToString(CRunnable):
    def process(self, data):
        print("Convert to string: ", data)
        return f"Result: {data}"

In [22]:
a = AddTen()
b = MultiplyByTwo()
c = ConvertToString()

chain = a | b | c

In [ ]:
result = chain.invoke(10)
print(result)

### Runnables from LangChain

In [24]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

In [ ]:
chain = RunnablePassthrough() | RunnablePassthrough () | RunnablePassthrough ()
chain.invoke("hello")

In [26]:
def input_to_upper(input: str):
    output = input.upper()
    return output

In [ ]:
chain = RunnablePassthrough() | RunnableLambda(input_to_upper) | RunnablePassthrough()
chain.invoke("hello")

In [29]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": RunnablePassthrough()})

In [ ]:
chain.invoke("hello")

In [ ]:
chain.invoke({"input": "hello", "input2": "goodbye"})

In [32]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": lambda z: z["input2"]})

In [ ]:
chain.invoke({"input": "hello", "input2": "goodbye"})

### Nested chains - now it gets more complicated!

In [34]:
def find_keys_to_uppercase(input: dict):
    output = input.get("input", "not found").upper()
    return output

In [35]:
chain = RunnableParallel({"x": RunnablePassthrough() | RunnableLambda(find_keys_to_uppercase), "y": lambda z: z["input2"]})

In [ ]:
chain.invoke({"input": "hello", "input2": "goodbye"})

In [41]:
chain = RunnableParallel({"x": RunnablePassthrough()})

def assign_func(_):
    return 100

def multiply(input):
    return input * 10

In [ ]:
chain.invoke({"input": "hello", "input2": "goodbye"})

In [43]:
chain = RunnableParallel({"x": RunnablePassthrough()}).assign(extra=RunnableLambda(assign_func))

In [ ]:
result = chain.invoke({"input": "hello", "input2": "goodbye"})
print(result)

### Combine multiple chains

In [45]:
def extractor(input: dict):
    return input.get("extra", "Key not found")

def cupper(upper: str):
    return str(upper).upper()

new_chain = RunnableLambda(extractor) | RunnableLambda(cupper)

In [ ]:
new_chain.invoke({"extra": "test"})

In [ ]:
final_chain = chain | new_chain
final_chain.invoke({"input": "hello", "input2": "goodbye"})